# Train

Algoritmi:

**K-Nearest Neighbors (KNN)**:
Pro: Semplice da implementare, non richiede addestramento costoso.
Contro: Prestazioni possono decadere con dataset di grandi dimensioni. Sensibile alla scala delle feature.

**Decision Trees**:
Pro: Facile da interpretare, gestisce automaticamente le feature rilevanti.
Contro: Tendenza all'overfitting. Può essere instabile con piccole variazioni nei dati.

**Linear Regression**:
Pro: Semplice e interpretabile, adatta per relazioni lineari tra feature e target.
Contro: Sensibile a outliers. Non gestisce bene relazioni complesse.

**Logistic Regression**:
Pro: Buona per problemi di classificazione binaria.
Contro: Assume una relazione lineare tra le feature e il log-odds. Non gestisce bene relazioni complesse.

**Support Vector Machines (SVM)**:
Pro: Buone prestazioni in spazi delle feature ad alta dimensione.
Contro: Richiede una scelta accurata dei parametri. Non sempre efficace con dataset molto grandi.

**Random Forest**:
Pro: Buona capacità di gestire complessità e overfitting. Può fornire importanza delle feature.
Contro: Meno interpretabile rispetto ai singoli alberi. Può richiedere tempo per l'addestramento.

**Ensemble Methods**:
Pro: Combina diversi modelli per migliorare le prestazioni complessive.
Contro: Complessità e interpretabilità possono essere un problema.

**Neural Networks**:
Pro: Eccellenti per problemi complessi e non lineari. Addestramento su grandi quantità di dati.
Contro: Richiede molto dati e risorse di calcolo. Complessità nella scelta dell'architettura.

**Convolutional Neural Networks (CNN)**:
Pro: Eccellenti per dati strutturati come immagini. Applicabili anche a dati sequenziali.
Contro: Richiede dati etichettati in grandi quantità.

Escludiamo quindi Linear Regression, Decision Trees,Support Vector Machines (SVM) e Convolutional Neural Networks (CNN).

Considerazioni:

Possiamo utilizzare i rimanenti algoritmi per dividere il problema il 4 categorie: Unsafe, Target 1 day, Target 5 days, Target 30 days.

**Knn**:
La scelta del K è importante e può variare molto prendendo due K molto vicini.
Per questo motivo abbiamo deciso di affidarci ad altri tipi di algoritmi per questo tipo di problema.
 
**Logistic Regression**: 
Possiamo utilizzare la Regressione Logistica per dividere il problema in 3 sottoproblemi. Possiamo classificare se le predizioni appartengono o meno all'etichetta Target 1 day, Target 5 days o Target 30 days. Se tutte e 3 dovessero essere 0, la scelta ricadrebbe su Unsafe.
Questo vorrebbe dire allenare 3 volte l'algoritmo predicendo una alla volta tutte e tre le etichette.
(Da testare)

**Random Forests**:
Possiamo utilizzare la tecnica del Bagging per ridurre la varianza, tipicamente elevata con questo algoritmo, per avere un accuracy più alta.
(Da testare)

**Ensemble Methods**: 
(Che metodi utilizzare?)
(Da Testare)

**Neural Networks**: 
L'utilizzo delle neural networks risulta essere più complicato degli altri poiché è più difficile da costruire e può svilupparsi in diverse varianti.
La parte difficile appunto sta nella scelta del numero di hidden layers e nel numero di nodi di questi. Tale scelta può essere affinata con test ripetuti per arrivare ad una soluzione comune.
(Da testare)